In [1]:
!wget https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized/blob/main/data/train_prefs-00000-of-00001-17309c769bfe5733.parquet
import pandas as pd
df = pd.read_parquet("train_prefs-00000-of-00001-17309c769bfe5733.parquet")

In [3]:
import numpy as np
from rwkv.utils import PIPELINE
from rwkv.model import RWKV

In [5]:
model = RWKV("D:/models/rwkv/RWKV-5-World-0.4B-v2-20231113-ctx4096.pth", "cuda bf16")

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading D:/models/rwkv/RWKV-5-World-0.4B-v2-20231113-ctx4096.pth ...
Model detected: v5.2
Strategy: (total 24+1=25 layers)
* cuda [bfloat16, bfloat16], store 25 layers
0-cuda-bfloat16-bfloat16 1-cuda-bfloat16-bfloat16 2-cuda-bfloat16-bfloat16 3-cuda-bfloat16-bfloat16 4-cuda-bfloat16-bfloat16 5-cuda-bfloat16-bfloat16 6-cuda-bfloat16-bfloat16 7-cuda-bfloat16-bfloat16 8-cuda-bfloat16-bfloat16 9-cuda-bfloat16-bfloat16 10-cuda-bfloat16-bfloat16 11-cuda-bfloat16-bfloat16 12-cuda-bfloat16-bfloat16 13-cuda-bfloat16-bfloat16 14-cuda-bfloat16-bfloat16 15-cuda-bfloat16-bfloat16 16-cuda-bfloat16-bfloat16 17-cuda-bfloat16-bfloat16 18-cuda-bfloat16-bfloat16 19-cuda-bfloat16-bfloat16 20-cuda-bfloat16-bfloat16 21-cuda-bfloat16-bfloat16 22-cuda-bfloat16-bfloat16 23-cuda-bfloat16-bfloat16 24-cuda-bfloat16-bfloat16 
emb.weight                       bf16      cpu  65536  1024 
blocks.0.ln1.weight              bf16   cuda:0   1024       
blocks.0.ln1.bias      

In [7]:
tokenizer = PIPELINE(model, "rwkv_vocab_v20230424")

In [8]:
tokenizer.encode("\n\nUser:"), tokenizer.encode("\n\nResponse:")

([261, 24281, 59], [261, 53671, 59])

In [8]:
import random
random.seed(100)
train_percent = 0.8
train_valid_percent = 0.9

In [9]:
trainset = []
validset = []
testset = []

In [10]:
import torch
import torch.nn.functional as F

In [11]:
def compute_base_prob(prompt_tokens, chosen_tokens):
    global tokenizer
    full_logits_chosen, _ = tokenizer.model.forward(prompt_tokens + chosen_tokens[:-1], None, full_output=True)
    chosen_logits = full_logits_chosen[-len(chosen_tokens):]
    chosen_loss = (F.log_softmax(chosen_logits, dim=-1))[torch.arange(len(chosen_tokens)), chosen_tokens]
    return float(torch.sum(chosen_loss))

In [12]:
for i in range(100):
    prompt_str = str(df.iloc[i].prompt).strip()
    chosen_str = str(df.iloc[i].chosen[1]["content"]).strip().replace("\n\n", "\n")
    reject_str = str(df.iloc[i].rejected[1]["content"]).strip().replace("\n\n", "\n")
    h = random.random()
    if h < 0.8:
        prompt_str = "User: " + prompt_str + "\n\nAssistant:" # Helpfulness optimization
        chosen_str = ' ' + chosen_str + "\n\n"
        reject_str = ' ' + reject_str + "\n\n"
    elif h<0.9:
        prompt_str = "Question: " + prompt_str + "\n\nAnswer:" # Factuality
        chosen_str = ' ' + chosen_str + "\n\n"
        reject_str = ' ' + reject_str + "\n\n"
    else:
        prompt_str = "Input: " + prompt_str + "\n\nResponse:" # Instruction-following
        chosen_str = ' ' + chosen_str + "\n\n"
        reject_str = ' ' + reject_str + "\n\n"
    prompt_tokens = tokenizer.encode(prompt_str)
    chosen_tokens = tokenizer.encode(chosen_str)
    reject_tokens = tokenizer.encode(reject_str)

    chosen_base_prob = compute_base_prob(prompt_tokens, chosen_tokens)
    reject_base_prob = compute_base_prob(prompt_tokens, reject_tokens)

    print(chosen_base_prob, reject_base_prob)
    # prompt_chosen_mask = [0] * (len(prompt_tokens)-1) + [1] * len(chosen_tokens)
    # prompt_reject_mask = [0] * (len(prompt_tokens)-1) + [1] * len(reject_tokens)
    h = random.random()
    if h < train_percent:
        trainset.append((prompt_tokens, chosen_tokens, reject_tokens, chosen_base_prob, reject_base_prob))
    elif h < train_valid_percent:
        validset.append((prompt_tokens, chosen_tokens, reject_tokens, chosen_base_prob, reject_base_prob))
    else:
        testset.append((prompt_tokens, chosen_tokens, reject_tokens, chosen_base_prob, reject_base_prob))

d:\Program Files\Python311\Lib\site-packages\rwkv\model.py:1138: UserWarning: operator () profile_node %157 : int = prim::profile_ivalue(%155)
 does not have profile information (Triggered internally at ..\third_party\nvfuser\csrc\graph_fuser.cpp:108.)
  x, state[i*3+0], state[i*3+1] = ATT(


-1845.47900390625 -768.1392822265625
-927.13330078125 -1809.7193603515625
-367.13763427734375 -184.26104736328125
-151.35646057128906 -566.4130859375
-532.8018798828125 -166.65216064453125
-909.8840942382812 -844.625732421875
-605.8050537109375 -33.850189208984375
-181.65533447265625 -92.8620376586914
-169.68191528320312 -398.36627197265625
-1132.143798828125 -482.20489501953125
-84.56585693359375 -41.13652801513672
-148.7683868408203 -19.554271697998047
-1841.4710693359375 -1194.8499755859375
-905.9949340820312 -375.1104736328125
-34.30028533935547 -25.615936279296875
-282.25030517578125 -200.34786987304688
-56.40519714355469 -105.28010559082031
-29.12896156311035 -73.40998077392578
-34.48490524291992 -450.10784912109375
-115.04679870605469 -16.300830841064453
-58.02155685424805 -215.5607147216797
-936.566162109375 -43.03139877319336
-85.80197143554688 -28.40114974975586
-7.36306619644165 -172.81602478027344
-445.5924072265625 -836.9470825195312
-79.41017150878906 -31.255094528198242


In [13]:
import torch
torch.save(trainset, "trainset.save")
torch.save(validset, "validset.save")
torch.save(testset, "testset.save")